## BASELINE CNN
- 思路：先做Jupyterbooks，有了运行结果后放进class跑。11号今天写CNN并跑出基本结果。
- train (5509260, 14) , validation (1836420, 14) , test (1836420, 14)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf

%matplotlib inline
plt.style.use('ggplot')


In [2]:
train_path = '../src/train.csv'
valid_path = '../src/validation.csv'
test_path = '../src/test.csv'
combined_path = '../src/combined_file.csv'

In [3]:
def read_data(file_path):
    # column_names = ['subject','activity','time_s', 'lw_x','lw_y','lw_z','lh_x','lh_y','lh_z','la_x','la_y','la_z','ra_x','ra_y','ra_z']
    columns_to_drop = ['lw_x', 'lw_y', 'lw_z', 'lh_x', 'lh_y', 'lh_z', 'la_x', 'la_y', 'la_z', 'ra_x', 'ra_y', 'ra_z']
    data = pd.read_csv(file_path,header = 0)
    # Drop the columns from the DataFrame
    df = data.drop(columns=columns_to_drop)
    return df

def plot_axis(ax, x, y, title):
    ax.plot(x, y)
    ax.set_title(title)
    ax.xaxis.set_visible(False)
    ax.set_ylim([min(y) - np.std(y), max(y) + np.std(y)])
    ax.set_xlim([min(x), max(x)])
    ax.grid(True)
    
def plot_activity(activity,data):
    fig, (ax0, ax1, ax2, ax3) = plt.subplots(nrows = 4, figsize = (15, 10), sharex = True)
    plot_axis(ax0, data['time_s'], data['magnitude_lw'], 'magnitude_lw')
    plot_axis(ax1, data['time_s'], data['magnitude_lh'], 'magnitude_lh')
    plot_axis(ax2, data['time_s'], data['magnitude_la'], 'magnitude_la')
    plot_axis(ax3, data['time_s'], data['magnitude_ra'], 'magnitude_ra')
    plt.subplots_adjust(hspace=0.2)
    fig.suptitle(activity)
    plt.subplots_adjust(top=0.90)
    plt.show()


In [4]:
train_reference_df = read_data(train_path)
train_df = train_reference_df.copy()
train_df = train_df
val_reference_df = read_data(valid_path)

In [5]:
# Enable smoke test
smoke_test = True
smoke_test_size = 50


if smoke_test:
    train_reference_df = train_reference_df[:smoke_test_size]

    val_reference_df = val_reference_df[:smoke_test_size]


In [6]:
train_reference_df.shape
column_names = ['subject','activity']
Y_train = train_reference_df['activity']
X_train = train_reference_df.drop(columns=column_names)

print(f'X_train shape is {X_train.shape} , Y_train shape is {Y_train.shape}')
# x_train = train_reference_df[column_names]


X_train shape is (50, 6) , Y_train shape is (50,)


In [7]:
from tensorflow.keras.utils import to_categorical

Y_train = Y_train - np.min(Y_train)  # Ensure labels start from 0
num_classes = len(np.unique(Y_train))

Y_train = to_categorical(Y_train, num_classes=num_classes)

num_classes

1

In [9]:
print(f'X_train shape is {X_train.shape} , Y_train shape is {Y_train.shape}')

X_train shape is (50, 6) , Y_train shape is (50, 1)


In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(6,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))  # replace num_classes with the number of classes



# Compile and train the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# # Convert your dataframe to numpy arrays for training
# X_train = X_train.values
# Y_train = Y_train.values



model.fit(X_train, Y_train, epochs=10)


Epoch 1/10
2/2 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 2/10
2/2 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 3/10
2/2 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 4/10
2/2 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 5/10
2/2 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 6/10
2/2 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 7/10
2/2 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 8/10
2/2 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 9/10
2/2 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 10/10
2/2 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 1.0000

/Users/menglutao/.pyenv/versions/3.8.10/envs/HDA/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:1176: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)
